<a href="https://colab.research.google.com/github/beatlesatani/coding_september/blob/main/web_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=5784c821c37cc15d46f716d219c3d868cd246b73d7e1f47fdf47a07fe4ac5ec1
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [2]:
!pip install flask-ngrok
!ngrok authtoken 2VuYeOSgGeuMZMXNzYdH4j5h23p_5jvjNn2J8vsFZSdGqYLu3

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
!pip install tensorflow_text
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.4 MB/s eta 0:00:00


In [4]:
from transformers import BertTokenizer,TFBertModel,BertConfig,TFBertForSequenceClassification
import tensorflow as tf
import torch
import numpy as np
from tensorflow import keras
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template
import pickle
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [6]:
model_path = '/drive/MyDrive/sentiment_analysis/bert_model.tf'
# load the model
loaded_model = keras.models.load_model(model_path)

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case = True)

In [8]:
test_sentence = ['i am so happy']

In [9]:
tf_batch = tokenizer(test_sentence, max_length = 75, padding=True, truncation=True, return_tensors='tf')

In [10]:
tf_outputs = loaded_model(tf_batch)

In [11]:
tf_outputs

{'logits': <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-2.0649917, -0.8902506,  3.5346541]], dtype=float32)>}

In [12]:
logits = tf_outputs['logits']

# Apply softmax to the logits to get probabilities
probabilities = tf.nn.softmax(logits, axis=-1)
print(probabilities)

# Get the predicted class with the highest probability
predicted_class = np.argmax(probabilities, axis=-1)

# The predicted_class variable now contains the predicted sentiment label (0, 1, or 2)
print("Predicted Sentiment:", predicted_class[0])

tf.Tensor([[0.00364209 0.01179054 0.9845674 ]], shape=(1, 3), dtype=float32)
Predicted Sentiment: 2


In [13]:
# i tried to use my original tokenizer from final.ipnyb.
# but by using this I got lots of error, like score is beyond the range of [0,1,2] like 402
""""def mask_inputs_for_bert(tweets, max_len):
  # Encode the tweet using the tokenizer.
  encoded_dict = tokenizer.encode_plus(
            tweets,
            add_special_tokens=True, #add cls and sep
            max_length=max_len,   #pad & truncate all sentences
            pad_to_max_length=True,
            return_attention_mask=True  #construct attention_masks
        )
  input_id = encoded_dict['input_ids']
  attention_mask =encoded_dict['attention_mask']

  # Convert the lists to tensors (vectors) and return
  input_id = tf.convert_to_tensor(input_id)
  attention_mask = tf.convert_to_tensor(attention_mask)

  return input_id, attention_mask"""

'"def mask_inputs_for_bert(tweets, max_len):\n  # Encode the tweet using the tokenizer.\n  encoded_dict = tokenizer.encode_plus(\n            tweets,\n            add_special_tokens=True, #add cls and sep\n            max_length=max_len,   #pad & truncate all sentences\n            pad_to_max_length=True,\n            return_attention_mask=True  #construct attention_masks\n        )\n  input_id = encoded_dict[\'input_ids\']\n  attention_mask =encoded_dict[\'attention_mask\']\n\n  # Convert the lists to tensors (vectors) and return\n  input_id = tf.convert_to_tensor(input_id)\n  attention_mask = tf.convert_to_tensor(attention_mask)\n\n  return input_id, attention_mask'

In [14]:
# put them into one function
def sentiment_analysis (text):
  tf_batch = tokenizer(text, max_length = 75, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = loaded_model(tf_batch)
  logits = tf_outputs['logits']
  # Apply softmax to the logits to get probabilities
  probabilities = tf.nn.softmax(logits, axis=-1)
  # Get the predicted class with the highest probability
  predicted_class = np.argmax(probabilities, axis=-1)
  if (predicted_class == 0):
    return 'Negative'
  elif (predicted_class == 1):
    return "Neutral"
  else:
    return "Positive"

In [15]:
text = "i'm eating apples"
print(sentiment_analysis(text))
#the funciton works

Neutral


In [16]:
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm

#this is a cord to invoke terminal

In [17]:
from flask import Flask, request, render_template
app = Flask(__name__, template_folder='/drive/MyDrive/templates')


run_with_ngrok(app)

def sentiment_analysis (text):
  tf_batch = tokenizer(text, max_length = 75, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = loaded_model(tf_batch)
  logits = tf_outputs['logits']
  # Apply softmax to the logits to get probabilities
  probabilities = tf.nn.softmax(logits, axis=-1)
  # Get the predicted class with the highest probability
  predicted_class = np.argmax(probabilities, axis=-1)
  if (predicted_class == 0):
    return 'Negative'
  elif (predicted_class == 1):
    return "Neutral"
  else:
    return "Positive"

# homepage
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        text = request.form['text']
        sentiment = sentiment_analysis(text)
        return render_template("index.html", text=text, sentiment=sentiment)
    else:
        # Handle GET request (initial page load) without sentiment analysis results
        return render_template("index.html")
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9d0d-34-138-51-149.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:17:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:17:28] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:17:40] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:17:49] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:17:58] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:18:42] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:18:51] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:21:29] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:21:48] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:21:56] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:23:00] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:23:09] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Sep/2023 19:23:18] "POST / HTTP/1.1" 200 -
INFO:werkzeug:12

In [17]:
tunnel_url = requests.get("http://localhost:4040/api/tunnels").text
j = json.loads(tunnel_url)
tunnel_url = j['tunnels'][0]['public_url']

In [ ]:
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '8050')


public_url

'http://5b28-34-138-51-149.ngrok-free.app'

In [19]:

ngrok.kill()
